#  Final Project: Etsy Marketplace Exploration

## Data Aquisition

In [1]:
# import HTML tools
from lxml import html
import xml.etree.ElementTree as ET

In [2]:
# read in the list of etsy page categories
import pandas as pd
etsy_pages = pd.read_csv("etsy_category_pages.csv")


In [3]:
page_cat = etsy_pages['page'].tolist()

In [4]:
# generate list of urls for each category

urls = []
start = "https://www.etsy.com/c/"

for x in page_cat:
    urls.append(start + x)

In [5]:
# All of these categories get a successful response 
# Now that we have a list of catgories with associated web pages, we can scrape subcategories
# After we collect subcategores we can get the shop names from each subcategory page

In [6]:
urls

['https://www.etsy.com/c/accessories',
 'https://www.etsy.com/c/bags-and-purses',
 'https://www.etsy.com/c/clothing',
 'https://www.etsy.com/c/shoes',
 'https://www.etsy.com/c/jewelry',
 'https://www.etsy.com/c/craft-supplies-and-tools',
 'https://www.etsy.com/c/weddings',
 'https://www.etsy.com/c/books-movies-and-music',
 'https://www.etsy.com/c/electronics-and-accessories',
 'https://www.etsy.com/c/toys-and-games',
 'https://www.etsy.com/c/art-and-collectibles',
 'https://www.etsy.com/c/bath-and-beauty',
 'https://www.etsy.com/c/home-and-living',
 'https://www.etsy.com/c/paper-and-party-supplies',
 'https://www.etsy.com/c/pet-supplies']

In [7]:
# Find all html elements which contain the links to the subcategory pages
import requests
import lxml
sub_elements = []

for url in urls:
    #send the request
    content = requests.get(url)
    #save the page source code to a string called content_string
    content_string = content.text.encode("utf-8")
    #pass the page source to our html parse
    doc = lxml.html.document_fromstring(content_string)
    # Find html element containing each subcategory
    element = doc.xpath("//li[@class='pb-xs-1 pl-xs-0']")
    sub_elements.append(element)

In [8]:
# Get urls for each of the subcategories and store them in a list
sub_links = []

for element in sub_elements:
    for x in element:
        child = x.getchildren()
        new = child[0].attrib['href']
        sub_links.append(new)

In [9]:
# Save subcategory links in a csv file
import csv

with open('subcategory_links.csv', 'w') as csvfile:
    fieldnames=['link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # iterate through category and subcategory
    for item in sub_links:
        writer.writerow({'link': item})

In [10]:
# open csv subcategory link file
# create new pandas dataframe called catdf

newdata = pd.read_csv("subcategory_links.csv",sep="/",names=('0','1','2','3','top_category','link_address'))
del(newdata['0'],newdata['1'],newdata['2'],newdata['3'])
catdf = newdata.dropna()


In [11]:
# extract subcatgory name from link and save as new csv file

def clean_links(text):
    return text.split("?")[0]

catdf['sub_category'] = catdf['link_address'].apply(clean_links)


/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
catdf.head()

,top_category,link_address,sub_category
1,accessories,hair-accessories?ref=catlink-219,hair-accessories
2,accessories,hats-and-caps?ref=catlink-25,hats-and-caps
3,accessories,baby-accessories?ref=catlink-1728,baby-accessories
4,accessories,scarves-and-wraps?ref=catlink-42,scarves-and-wraps
5,accessories,keychains-and-lanyards?ref=catlink-164,keychains-and-lanyards


In [13]:
# Function to get urls for pages 2-250 in each subcategory
# https://www.etsy.com/c/jewelry/bracelets?page=24

def get_page2(row):
    urls = []
    start = "https://www.etsy.com/c/"
    for i in range(2,250):
        urls.append(start + row['top_category'] + "/" + row['sub_category'] + "?page=" + str(i))
    return ",".join(urls)

In [14]:
# Apply function and story multiple page urls in column called page_nums

catdf['page_nums'] = catdf.apply(get_page2, axis=1)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42

/Users/jill/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
catdf.head()

,top_category,link_address,sub_category,page_nums
1,accessories,hair-accessories?ref=catlink-219,hair-accessories,https://www.etsy.com/c/accessories/hair-access...
2,accessories,hats-and-caps?ref=catlink-25,hats-and-caps,https://www.etsy.com/c/accessories/hats-and-ca...
3,accessories,baby-accessories?ref=catlink-1728,baby-accessories,https://www.etsy.com/c/accessories/baby-access...
4,accessories,scarves-and-wraps?ref=catlink-42,scarves-and-wraps,https://www.etsy.com/c/accessories/scarves-and...
5,accessories,keychains-and-lanyards?ref=catlink-164,keychains-and-lanyards,https://www.etsy.com/c/accessories/keychains-a...


In [16]:
sub_links = pd.read_csv("subcategory_links.csv")

In [17]:
# join dataframes

catdf = catdf.reset_index()
catdf = catdf.join(sub_links)
del catdf['index']
del catdf['link_address']

catdf.head(5)

,top_category,sub_category,page_nums,link
0,accessories,hair-accessories,https://www.etsy.com/c/accessories/hair-access...,https://www.etsy.com/c/accessories/hair-access...
1,accessories,hats-and-caps,https://www.etsy.com/c/accessories/hats-and-ca...,https://www.etsy.com/c/accessories/hats-and-ca...
2,accessories,baby-accessories,https://www.etsy.com/c/accessories/baby-access...,https://www.etsy.com/c/accessories/baby-access...
3,accessories,scarves-and-wraps,https://www.etsy.com/c/accessories/scarves-and...,https://www.etsy.com/c/accessories/scarves-and...
4,accessories,keychains-and-lanyards,https://www.etsy.com/c/accessories/keychains-a...,https://www.etsy.com/c/accessories/keychains-a...


In [18]:
# finds all shop names from html source string and appends to list
# returns string
import time

def get_shop_names(some_link):
    time.sleep(2)
    content = some_link
    source_code = requests.get(content)
    source_string = source_code.text.encode("utf-8")
    j = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//div[@class='card-meta-row-item text-truncate overflow-hidden card-shop-name']")
    for tag in tags:
        j.append(tag.text)
    return ",".join(j)

In [19]:
new_column = sub_links['link'].apply(get_shop_names)

In [20]:
catdf['raw_shops'] = new_column

In [21]:
# turn string into list of strings

def link_list(a_string):
    foo = a_string.split(",")
    return foo

In [24]:
catdf.head()

,top_category,sub_category,page_nums,link,raw_shops,more_shops
0,accessories,hair-accessories,https://www.etsy.com/c/accessories/hair-access...,https://www.etsy.com/c/accessories/hair-access...,\n LeChaim\n ...,
1,accessories,hats-and-caps,https://www.etsy.com/c/accessories/hats-and-ca...,https://www.etsy.com/c/accessories/hats-and-ca...,\n BeanieVille\n ...,
2,accessories,baby-accessories,https://www.etsy.com/c/accessories/baby-access...,https://www.etsy.com/c/accessories/baby-access...,\n LePetiteBirdie\n...,
3,accessories,scarves-and-wraps,https://www.etsy.com/c/accessories/scarves-and...,https://www.etsy.com/c/accessories/scarves-and...,\n fAtoZ\n ...,
4,accessories,keychains-and-lanyards,https://www.etsy.com/c/accessories/keychains-a...,https://www.etsy.com/c/accessories/keychains-a...,\n artbymorgie\n ...,


In [23]:
catdf['more_shops'] = ""

In [28]:
catdf['even_more_shops'] = ""

In [29]:
# run get_shop_names on page_nums
# j is length of catdf
# i is page of results you want to see (up to 250)

for j in range(0,161):
    trial = link_list(catdf['page_nums'][j])
    k = []
    for i in range(20,25):
        one = get_shop_names(trial[i])
        k.append(one)
    catdf['even_more_shops'][j] = ",".join(k)

In [30]:
catdf

,top_category,sub_category,page_nums,link,raw_shops,more_shops,even_more_shops
0,accessories,hair-accessories,https://www.etsy.com/c/accessories/hair-access...,https://www.etsy.com/c/accessories/hair-access...,\n LeChaim\n ...,\n TheIvoryWillowCo...,\n sprinklesforspro...
1,accessories,hats-and-caps,https://www.etsy.com/c/accessories/hats-and-ca...,https://www.etsy.com/c/accessories/hats-and-ca...,\n BeanieVille\n ...,\n RockyMountainHat...,\n CocoBridalBoutiq...
2,accessories,baby-accessories,https://www.etsy.com/c/accessories/baby-access...,https://www.etsy.com/c/accessories/baby-access...,\n LePetiteBirdie\n...,\n SkylarnMe\n ...,\n fashionfairytale...
3,accessories,scarves-and-wraps,https://www.etsy.com/c/accessories/scarves-and...,https://www.etsy.com/c/accessories/scarves-and...,\n fAtoZ\n ...,\n MiracleShine\n ...,\n MyKnitCroch\n ...
4,accessories,keychains-and-lanyards,https://www.etsy.com/c/accessories/keychains-a...,https://www.etsy.com/c/accessories/keychains-a...,\n artbymorgie\n ...,\n artbymorgie\n ...,\n artbymorgie\n ...
5,accessories,suit-and-tie-accessories,https://www.etsy.com/c/accessories/suit-and-ti...,https://www.etsy.com/c/accessories/suit-and-ti...,\n JewelMango\n ...,\n BridalCloset\n ...,\n BabyBouteek\n ...
6,accessories,patches-and-pins,https://www.etsy.com/c/accessories/patches-and...,https://www.etsy.com/c/accessories/patches-and...,\n Heartificial\n ...,\n dannybrito\n ...,\n LinnTwins\n ...
7,accessories,belts-and-suspenders,https://www.etsy.com/c/accessories/belts-and-s...,https://www.etsy.com/c/accessories/belts-and-s...,\n MelodyOfCuteness...,\n RhythmicMetal\n ...,\n bakerboyShop\n ...
8,accessories,costume-accessories,https://www.etsy.com/c/accessories/costume-acc...,https://www.etsy.com/c/accessories/costume-acc...,\n FutureHeroes\n ...,\n Merimask\n ...,\n Wintercroft\n ...
9,accessories,sunglasses-and-eyewear,https://www.etsy.com/c/accessories/sunglasses-...,https://www.etsy.com/c/accessories/sunglasses-...,\n SundryAccessorie...,\n SunglassMuseum\n...,\n CherryTreeLaneDe...


In [31]:
catdf.to_csv(path_or_buf="shop_names")

UnicodeEncodeError: 'ascii' codec can't encode character u'\xa0' in position 2942: ordinal not in range(128)

In [61]:
# takes raw string containing shop names
#returns cleaned with shop name only separated by comas

import re

def clean_name(messy_name):
    new_list = []
    for name in messy_name:
        finder = re.compile("\n" + "(.*?)" + "\n", re.IGNORECASE)
        matches = finder.findall(name)
        no_replace = re.compile("\s+")
        p = no_replace.sub("", matches[0])
        new_list.append(p)
    return ",".join(new_list)

In [33]:
catdf['all_shops'] = ""

In [78]:
catdf.tail(83)

,top_category,sub_category,page_nums,link,raw_shops,more_shops,even_more_shops,all_shops
78,weddings,clothing,https://www.etsy.com/c/weddings/clothing?page=...,https://www.etsy.com/c/weddings/clothing?ref=c...,\n justDressAndChar...,\n BrideAndEntourag...,\n AlexDress\n ...,"BrideAndEntourage,OneFancyDay,FrenchKnotCoutur..."
79,weddings,shoes,"https://www.etsy.com/c/weddings/shoes?page=2,h...",https://www.etsy.com/c/weddings/shoes?ref=catl...,\n bosphorusshop\n ...,\n Parisxox\n ...,\n walkinonair\n ...,"Parisxox,gorgeousweddingshoes,NewBrideCo,Pink2..."
80,books-movies-and-music,books,https://www.etsy.com/c/books-movies-and-music/...,https://www.etsy.com/c/books-movies-and-music/...,\n Graphecx\n ...,\n PaperCalling\n ...,\n CaseGears\n ...,"PaperCalling,MisterScribbles,CocoaPaper,planne..."
81,books-movies-and-music,music,https://www.etsy.com/c/books-movies-and-music/...,https://www.etsy.com/c/books-movies-and-music/...,\n PickslaysWoodwor...,\n SymetriColour\n ...,\n MaxProduction\n ...,"SymetriColour,DolmageDesigns,hornandcastle,Lyr..."
82,books-movies-and-music,movies,https://www.etsy.com/c/books-movies-and-music/...,https://www.etsy.com/c/books-movies-and-music/...,\n TheArcCosplay\n ...,\n PLukaszewskiArt\...,\n PhatThrift\n ...,"PLukaszewskiArt,BretagneBrocante,CeruleanSea23..."
83,books-movies-and-music,video-cases-and-tins,https://www.etsy.com/c/books-movies-and-music/...,https://www.etsy.com/c/books-movies-and-music/...,\n nauli\n ...,\n JaysFashion44\n ...,",,,,","JaysFashion44,StewvilleVintageShop,songs4youro..."
84,electronics-and-accessories,electronics-cases,https://www.etsy.com/c/electronics-and-accesso...,https://www.etsy.com/c/electronics-and-accesso...,\n mtldecals\n ...,\n WiseStitch\n ...,\n iPhoneCaseLove\n...,"WiseStitch,MissPurpleVelvet,ZiZZDess,LunaSavit..."
85,electronics-and-accessories,decals-and-skins,https://www.etsy.com/c/electronics-and-accesso...,https://www.etsy.com/c/electronics-and-accesso...,\n mtldecals\n ...,\n MoonAndStarCo\n ...,\n PrettyLettersSho...,"MoonAndStarCo,mtcvinyl,SunshineVinyl,SkinLee,V..."
86,electronics-and-accessories,car-parts-and-accessories,https://www.etsy.com/c/electronics-and-accesso...,https://www.etsy.com/c/electronics-and-accesso...,\n GlitterGlamMonog...,\n DarkStarLab\n ...,\n ChicMonogram\n ...,"DarkStarLab,GlitterGlamMonograms,AMillionMonog..."
87,electronics-and-accessories,cell-phone-accessories,https://www.etsy.com/c/electronics-and-accesso...,https://www.etsy.com/c/electronics-and-accesso...,\n Caseity\n ...,\n FunCasesUK\n ...,\n AnkaSupplyCo\n ...,"FunCasesUK,LovelyCaseCo,Animusrhythm,CloudsofA..."


In [98]:
catdf['even_more_shops'][99]

'\n                            PickersWarehouse\n                        ,\n                            ShoppingLounge\n                        ,\n                            VintageInquisitor\n                        ,\n                            silverliningtoys\n                        ,\n                            JennieJamesResale\n                        ,\n                            WhiteEnterprises\n                        ,\n                            Holodukes\n                        ,\n                            RetroToysAndMore\n                        ,\n                            JERSEYDEN\n                        ,\n                            TheJamesMarket\n                        ,\n                            AlphaDesignsKC\n                        ,\n                            PickinJoefromthebigO\n                        ,\n                            ShoppingLounge\n                        ,\n                            ShoppingLounge\n                    

In [120]:
s = catdf['more_shops'][160]

In [121]:
s

'\n                            GorillaDecals\n                        ,\n                            TOUGHTIMBERS\n                        ,\n                            LittleLarueApiary\n                        ,\n                            GalleryintheGarden\n                        ,\n                            VillageVinyl\n                        ,\n                            TOUGHTIMBERS\n                        ,\n                            VillageVinyl\n                        ,\n                            VillageVinyl\n                        ,\n                            TOUGHTIMBERS\n                        ,\n                            TOUGHTIMBERS\n                        ,\n                            VillageVinyl\n                        ,\n                            GalleryintheGarden\n                        ,\n                            GorillaDecals\n                        ,\n                            VillageVinyl\n                        ,\n            

In [122]:
catdf['even_more_shops'][160] = s

In [77]:
# clean more_shops and store names in all_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['more_shops'][i])
    catdf['all_shops'][i] = clean_name(bar)

In [79]:
catdf['all2_shops'] = ""

In [125]:
catdf['all3_shops'] = ""

In [123]:
# clean even_more_shops and store names in all_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['even_more_shops'][i])
    catdf['all2_shops'][i] = clean_name(bar)

In [126]:
# clean raw_shops and store names in all3_shops
# run 0-161

for i in range(0,161):
    bar = link_list(catdf['raw_shops'][i])
    catdf['all3_shops'][i] = clean_name(bar)

In [127]:
catdf.tail(3)

,top_category,sub_category,page_nums,link,raw_shops,more_shops,even_more_shops,all_shops,all2_shops,all3_shops
158,pet-supplies,pet-storage,https://www.etsy.com/c/pet-supplies/pet-storag...,https://www.etsy.com/c/pet-supplies/pet-storag...,\n TheTexasFarmhous...,\n Yarramalong\n ...,\n DogDesignsByDani...,"Yarramalong,BridgewoodPlace,Goods4Us,EtcherGir...","DogDesignsByDanielle,SammieJosDogProducts,King...","TheTexasFarmhouse,KingsBenchCreations,WonkaWor..."
159,pet-supplies,pet-gates-and-fences,https://www.etsy.com/c/pet-supplies/pet-gates-...,https://www.etsy.com/c/pet-supplies/pet-gates-...,\n MarkedSoles\n ...,\n KeepYourChinUp\n...,\n KeepYourChinUp\n...,"KeepYourChinUp,lindakratz,A1Decals,lindakratz,...","KeepYourChinUp,lindakratz,A1Decals,lindakratz,...","MarkedSoles,FusionGates,ScaleAndTailor,SignMac..."
160,pet-supplies,beekeeping,https://www.etsy.com/c/pet-supplies/beekeeping...,https://www.etsy.com/c/pet-supplies/beekeeping...,\n TOUGHTIMBERS\n ...,\n GorillaDecals\n ...,\n GorillaDecals\n ...,"GorillaDecals,TOUGHTIMBERS,LittleLarueApiary,G...","GorillaDecals,TOUGHTIMBERS,LittleLarueApiary,G...","TOUGHTIMBERS,GalleryintheGarden,AOKAFarm,Loshc..."


In [128]:
def appendcols(row):
    s = row['all_shops'] + row['all_shops'] + row['all_shops']
    return s

In [130]:
catdf['shop_names'] = ""

In [131]:
catdf['shop_names'] = catdf.apply(appendcols, axis=1)

In [133]:
catdf['all_shops'][2] == catdf['all2_shops'][2]

False

In [138]:
final_catdf = catdf[['top_category','sub_category','shop_names']]
#df2 = df[["col1", "col2", "col3"]]

In [226]:
s = final_catdf['shop_names'][121]

In [228]:
ss = s.encode("utf-8")

In [229]:
type(ss)

str

In [231]:
final_catdf['shop_names'][121] = ss

In [235]:
for i in range(120,161):
    print i
    print type(final_catdf['shop_names'][i])

120
<type 'str'>
121
<type 'str'>
122
<type 'str'>
123
<type 'str'>
124
<type 'str'>
125
<type 'str'>
126
<type 'str'>
127
<type 'str'>
128
<type 'str'>
129
<type 'str'>
130
<type 'str'>
131
<type 'str'>
132
<type 'str'>
133
<type 'str'>
134
<type 'str'>
135
<type 'str'>
136
<type 'str'>
137
<type 'str'>
138
<type 'str'>
139
<type 'str'>
140
<type 'str'>
141
<type 'str'>
142
<type 'str'>
143
<type 'str'>
144
<type 'str'>
145
<type 'str'>
146
<type 'str'>
147
<type 'str'>
148
<type 'str'>
149
<type 'str'>
150
<type 'str'>
151
<type 'str'>
152
<type 'str'>
153
<type 'str'>
154
<type 'str'>
155
<type 'str'>
156
<type 'str'>
157
<type 'str'>
158
<type 'str'>
159
<type 'str'>
160
<type 'str'>


In [236]:
final_catdf.to_csv(path_or_buf="shop_names")

In [238]:
final_catdf

,top_category,sub_category,shop_names
0,accessories,hair-accessories,"TheIvoryWillowCo,ChicBlue,TheBridesStore,Lovel..."
1,accessories,hats-and-caps,"RockyMountainHatCo,RambleBabyBoutique,Rawsells..."
2,accessories,baby-accessories,"SkylarnMe,SweetAndStitched,LacoHandMade,Simply..."
3,accessories,scarves-and-wraps,"MiracleShine,ScarfClub,Zojanka,LTCoutureAtelie..."
4,accessories,keychains-and-lanyards,"artbymorgie,accessoriesbyjamiep,NiOuiNinoucheG..."
5,accessories,suit-and-tie-accessories,"BridalCloset,LoveVanillaDew,ElegantCuff,TieObs..."
6,accessories,patches-and-pins,"dannybrito,ilikeCATSshop,DistinctDesignsUnltd,..."
7,accessories,belts-and-suspenders,"RhythmicMetal,ISHAOR,armoniia,TheLeatherBoy,Th..."
8,accessories,costume-accessories,"Merimask,superheroforparty,Customownnecklace,K..."
9,accessories,sunglasses-and-eyewear,"SunglassMuseum,retroandme,Propwood,CarvedAcces..."


In [241]:
every_shop = []

for i in range(len(final_catdf['shop_names'])):
    something = link_list(catdf['shop_names'][i])
    for x in something:
        every_shop.append(x)

In [242]:
shop_df = pd.DataFrame(data = every_shop)

In [243]:
shop_df['shop_name'] = shop_df[0]

In [244]:
shop_df

,0,shop_name
0,TheIvoryWillowCo,TheIvoryWillowCo
1,ChicBlue,ChicBlue
2,TheBridesStore,TheBridesStore
3,LovelyLittlesandCo,LovelyLittlesandCo
4,masonandharlow,masonandharlow
5,RoyalSistersBoutique,RoyalSistersBoutique
6,LeftbankSV,LeftbankSV
7,avtboutique,avtboutique
8,thomaspark,thomaspark
9,buttercupsbows,buttercupsbows


In [223]:
shop_df['review_links'] = ""

In [ ]:
len(shop_df)

In [224]:
# generate urls for each shop name in the list

for i in range(len(shop_df)):
    x = shop_df['shop_name'][i]
    url = ("https://www.etsy.com/shop/"+ x +"/reviews")
    shop_df['review_links'][i] = url

In [228]:
shop_df.head(5)

,0,shop_name,review_links
0,KinseyGraceCo,KinseyGraceCo,https://www.etsy.com/shop/KinseyGraceCo/reviews
1,APPLEMINTHOUSE,APPLEMINTHOUSE,https://www.etsy.com/shop/APPLEMINTHOUSE/reviews
2,YogaHeadbands,YogaHeadbands,https://www.etsy.com/shop/YogaHeadbands/reviews
3,PRIVATEBRIDES,PRIVATEBRIDES,https://www.etsy.com/shop/PRIVATEBRIDES/reviews
4,PrettyandPoshCouture,PrettyandPoshCouture,https://www.etsy.com/shop/PrettyandPoshCouture...


In [226]:
# function to get list of item review dates from each shop review page

def get_shop_dates(some_link):
    time.sleep(2)
    content = some_link
    source_code = requests.get(content)
    source_string = source_code.text.encode("utf-8")
    
    p_elements = []
    a_elements = []
    list_dates = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//div[@class='mt-xs-2 mb-xs-2']")
    
    for tag in tags:
        p_elements.append(tag.getchildren())
    for p in p_elements:
        for x in p:
            a_elements.append(x.getchildren())
    for element in a_elements:
        for x in element:
            list_dates.append(x.tail)
    return ",".join(list_dates)

In [233]:
shop_df['review_dates'] = ""

In [247]:
shop_df[100:120]

,0,shop_name,review_links,review_dates
100,CandyZombies,CandyZombies,https://www.etsy.com/shop/CandyZombies/reviews,"on Aug 17, 2016\n , on Jul..."
101,Kamipapa,Kamipapa,https://www.etsy.com/shop/Kamipapa/reviews,"on Aug 20, 2016\n , on Aug..."
102,TrueNorthCollection,TrueNorthCollection,https://www.etsy.com/shop/TrueNorthCollection/...,"on Aug 20, 2016\n , on Aug..."
103,ChicBlue,ChicBlue,https://www.etsy.com/shop/ChicBlue/reviews,"on Aug 19, 2016\n , on Aug..."
104,LuluSplendor,LuluSplendor,https://www.etsy.com/shop/LuluSplendor/reviews,"on Aug 20, 2016\n , on Aug..."
105,GemsAndBones,GemsAndBones,https://www.etsy.com/shop/GemsAndBones/reviews,"on Aug 20, 2016\n , on Aug..."
106,buttercupsbows,buttercupsbows,https://www.etsy.com/shop/buttercupsbows/reviews,"on Aug 20, 2016\n , on Aug..."
107,ComicGeekOut,ComicGeekOut,https://www.etsy.com/shop/ComicGeekOut/reviews,"on Aug 20, 2016\n , on Aug..."
108,FitNorthwest,FitNorthwest,https://www.etsy.com/shop/FitNorthwest/reviews,"on Aug 20, 2016\n , on Aug..."
109,CrystalEclipseCrowns,CrystalEclipseCrowns,https://www.etsy.com/shop/CrystalEclipseCrowns...,"on Aug 17, 2016\n , on Aug..."


In [244]:
for i in range(80,500):
    new_column = get_shop_dates(shop_df['review_links'][i])
    shop_df['review_dates'][i] = new_column

In [ ]:
shop_df['review_dates'] = ""

In [ ]:
# populate column with list of review dates
# apply function to first 200 shops

for i in range(200):
    shop_df['review_dates'][i] = get_shop_dates(shop_df['review_source'][i])

In [ ]:
# function to get list of item ratings from each shop review page

def get_review_ratings(source_string):
    span_elements = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//span[@class='screen-reader-only']")
    
    for tag in tags:
        span_elements.append(tag.text)
    return span_elements

In [ ]:
shop_df['ratings'] = ""

In [ ]:
# populate column with list of review ratings
# apply function to first 200 shops

for i in range(200):
    shop_df['ratings'][i] = get_review_ratings(shop_df['review_source'][i])

In [ ]:
# function to get list of item review decriptions from each shop review page

def get_shop_details(source_string):
    p_elements = []
    a_elements = []
    list_details = []
    doc = lxml.html.document_fromstring(source_string)
    tags = doc.xpath("//div[@class='flag-body hide-xs hide-sm']")
    
    for tag in tags:
        p_elements.append(tag.getchildren())
    for p in p_elements:
        for x in p:
            a_elements.append(x.getchildren())
    for a in a_elements:
        for x in a:
            list_details.append(x.text)
    return list_details

In [ ]:
shop_df['item_details'] = ""

In [ ]:
# populate column with list of review descriptions
# apply function to first 200 shops

for i in range(200):
    shop_df['item_details'][i] = get_shop_details(shop_df['review_source'][i])

In [ ]:
shop_df.head(10)

In [ ]:
shopdata = shop_df

In [ ]:
del shopdata['review_source']

In [ ]:
shopdata

In [ ]:
shopdata.to_csv(path_or_buf = "shop_data")